### Notebook for testing and validating Reverify implementation

In [1]:
Pkg.update() 
Pkg.add("JuMP")
Pkg.add("GLPKMathProgInterface")

INFO: Updating METADATA...
INFO: Computing changes...
INFO: No packages to install, update or remove
INFO: Package JuMP is already installed
INFO: Package GLPKMathProgInterface is already installed


In [2]:
using JuMP
using MathProgBase
using GLPKMathProgInterface

In [3]:
include("../utils/util.jl")
include("../utils/problem.jl")
include("../utils/activation.jl")

LoadError: LoadError: LoadError: LoadError: LoadError: [91mUndefVarError: GeneralActivation not defined[39m
while loading /home/amelia/NeuralVerification.jl/utils/activation.jl, in expression starting on line 9
while loading /home/amelia/NeuralVerification.jl/utils/network.jl, in expression starting on line 1
while loading /home/amelia/NeuralVerification.jl/utils/problem.jl, in expression starting on line 2
while loading /home/amelia/NeuralVerification.jl/utils/util.jl, in expression starting on line 1

In [ ]:
function add_input_constraints(nnet::Network, m::Model, upperBounds::Vector{Float64}, lowerBounds::Vector{Float64}, 
                               A::Matrix{Float64} = [eye(length(upperBounds)); -eye(length(lowerBounds))])
    n_inputs = size(nnet.layers[1].weights)[2]
    b = [upperBounds; -lowerBounds]
    x_in = @variable(m, [1:n_inputs])
    @constraint(m, A*x_in .<= b)
    return x_in
end

In [ ]:
function add_output_constraints(nnet::Network, m::Model, upperBounds::Vector{Float64}, lowerBounds::Vector{Float64}, 
                                A::Matrix{Float64}=[eye(length(upperBounds)); -eye(length(lowerBounds))])
    n_outputs = size(nnet.layers[length(nnet.layers)].weights)[1]
    b = [upperBounds; -lowerBounds]
    x_out = @variable(m, [1:n_outputs])
    @constraint(m, A * x_out .<= b)
    return x_out
end

In [ ]:
function add_network_constraints(nnet::Network, m::Model,  x_in::Array{JuMP.Variable,1}, 
                                 x_out::Array{JuMP.Variable,1}, M::Float64)
    layers = nnet.layers
    x_net = @variable(m, [i=1:length(layers) - 1, j=1:length(layers[i].bias)]) 
    deltas = @variable(m, [i=1:length(layers), j=1:length(layers[i].bias)], Bin) 
    
    for i in 1:length(layers) - 1
        weights = layers[i].weights
        bias = layers[i].bias
        for j in 1:length(bias)
            if i == 1
                dot_prod = @expression(m, [k=1:size(nnet.layers[1].weights)[2]], sum(weights[j,k] * x_in[k]))
            else
                dot_prod = @expression(m, [k=1:length(layers[i-1].bias)], sum(weights[j,k] * x_net[i-1,k]))
            end
            @constraint(m, x_net[i,j] .>= dot_prod + bias[j]) 
            @constraint(m, x_net[i,j] .<= dot_prod + bias[j] + M*deltas[i,j])
            @constraint(m, x_net[i,j] >= 0)
            @constraint(m, x_net[i,j] <= M*(1 - deltas[i,j]))
        end
    end
    
    i = length(layers)
    weights = layers[i].weights
    bias = layers[i].bias
    for l in 1:length(bias)
        k_upper = length(nnet.layers[i-1].bias)
        dot_prod = @expression(m, [k=1:length(nnet.layers[i-1].bias)], sum(weights[l,k] * x_net[i-1,k]))
        @constraint(m, x_out[l] .>= dot_prod + bias[l])
        @constraint(m, x_out[l] .<= dot_prod + bias[l] + M*deltas[i,l])
        @constraint(m, x_out[l] >= 0)
        @constraint(m, x_out[l] <= M*(1 - deltas[i,l]))
    end
end

## Small Network Example
* small network has two hidden layers with 2 neurons and input and output layers with 1 neuron
* for input 0.0, small_nnet should produce output 55.0 

In [ ]:
# Read in small_nnet
small_nnet = read_nnet("../examples/networks/small_nnet.txt")

In [ ]:
input = [0.0]
compute_output(small_nnet, input)

In [ ]:
# Should return optimal
test_model = Model(solver =GLPKSolverMIP())
input_lower_bounds = [0.0]
input_upper_bounds = [0.0]
x_in = add_input_constraints(small_nnet, test_model, input_upper_bounds, input_lower_bounds)
output_lower_bounds = [0.0]
output_upper_bounds = [54.5]
x_out = add_output_constraints(small_nnet, test_model, output_upper_bounds, output_lower_bounds)
M = 55.0
add_network_constraints(small_nnet, test_model, x_in, x_out, M)
status = solve(test_model)
println(status)

In [ ]:
# Should return infeasible
test_model = Model(solver = GLPKSolverMIP())
input_lower_bounds = [0.0]
input_upper_bounds = [0.0]
x_in = add_input_constraints(small_nnet, test_model, input_upper_bounds, input_lower_bounds)
output_lower_bounds = [55.0]
output_upper_bounds = [55.0]
x_out = add_output_constraints(small_nnet, test_model, output_upper_bounds, output_lower_bounds)
M = 55.0
add_network_constraints(small_nnet, test_model, x_in, x_out, M)
status = solve(test_model)
println(status)

## IPCP Example
* IPCP network has three hidden layers of size 16
* Input layer has 4 neurons and output layer has 2 neurons
* The following test cases are those described by the Reverify paper
* NOTE: validation is pending M values used by Lomuscio et al.

In [ ]:
# Read in ipcp_nnet from code provided by Professor Lomuscio
ipcp_nnet = read_nnet("../examples/networks/cartpole_nnet.txt")
layers = ipcp_nnet.layers
println(layers[1])

In [ ]:
function add_ipcp_output_constraints(ipcp_model, n)
    x_out = @variable(ipcp_model, [1:2])
    @constraint(ipcp_model, x_out[1] >= x_out[2] + n)
    return x_out
end

In [ ]:
# IPCP example one- should return Infeasible
ipcp_model = Model(solver = GLPKSolverMIP())
input_lower_bounds = [0.0, 0.0, -5.0, 0.0]
input_upper_bounds = [0.0, 0.0, -5.0, 0.0]
x_in = add_input_constraints(ipcp_nnet, ipcp_model, input_upper_bounds, input_lower_bounds)
x_out = add_ipcp_output_constraints(ipcp_model, 100.0)
M = 1000.0
add_network_constraints(ipcp_nnet, ipcp_model, x_in, x_out, M)
status = solve(ipcp_model)
println(status)

In [ ]:
# IPCP example two - should return Infeasible
ipcp_model = Model(solver = GLPKSolverMIP())
input_lower_bounds = [-0.5,-0.2,-5,-0.1]
input_upper_bounds = [0.5,0.2,-4.0,-0.1]
x_in = add_input_constraints(ipcp_nnet, ipcp_model, input_upper_bounds, input_lower_bounds)
x_out = add_ipcp_output_constraints(ipcp_model, 100.0)
M = 1000.0
add_network_constraints(ipcp_nnet, ipcp_model, x_in, x_out, M)
status = solve(ipcp_model)
print(status)

In [ ]:
# IPCP example three - should return Infeasible
ipcp_model = Model(solver = GLPKSolverMIP())
input_lower_bounds = [-2.0,-0.2,-2.0,-0.25]
input_upper_bounds = [2.0,0.2,-1.5,0.25]
x_in = add_input_constraints(ipcp_nnet, ipcp_model, input_upper_bounds, input_lower_bounds)
x_out = add_ipcp_output_constraints(ipcp_model, 10.0)
M = 1000.0
add_network_constraints(ipcp_nnet, ipcp_model, x_in, x_out, M)
status = solve(ipcp_model)
println(status)

In [ ]:
# IPCP example four - should return Optimal
ipcp_model = Model(solver = GLPKSolverMIP())
input_lower_bounds = [-2.0,-0.4,-2.0,0.0]
input_upper_bounds = [2.0,0.4,1.0,0.1]
x_in = add_input_constraints(ipcp_nnet, ipcp_model, input_upper_bounds, input_lower_bounds)
x_out = add_ipcp_output_constraints(ipcp_model, 10.0)
M = 1000.0
add_network_constraints(ipcp_nnet, ipcp_model, x_in, x_out, M)
status = solve(ipcp_model)
println(status)